In [7]:
import duckdb
import pyarrow.dataset as ds
import pandas as pd
import tempfile

In [2]:
dataset = ds.dataset("data/status", format="parquet", partitioning="hive")

In [3]:
con = duckdb.connect()
con = con.register("status", dataset)

In [4]:
station_ids = con.execute("select distinct(station_id) from status").df()["station_id"].values
len(station_ids)

472

## Single station

### Base select + union select with leads

In [7]:
%%time 
for station_id in station_ids[:1]:
    df_query = f"""
    WITH base_status AS (select
        station_id,
        hour,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        status,
        make_timestamp(year, month, day, hour, minute, 0.0) as ts,
    from
        status
    where
        year = 2022 and
        month = 10 and
        station_id = {station_id} and
        status = 'IN_SERVICE')"""
    df_query += " union ".join([
    f"""
    select
        station_id,
        hour,
        dayofweek(ts) as dow,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        minute(lead(ts, {i}) over (
            order by ts asc
        ) - ts)  as minutes_bt_check,
        lead(num_bikes_available, {i}) over (
            order by ts asc
        ) as remaining_bikes_available,
    from
        base_status
    """ for i in list(range(1, 7)) + list(range(7, 18, 3))])
    df1 = con.execute(df_query).df().dropna()

print(f"dataset size {len(df1)}")

dataset size 36662
CPU times: user 19.9 s, sys: 766 ms, total: 20.7 s
Wall time: 3.32 s


### Base select with leads + union select per lead

In [8]:
%%time 
for station_id in station_ids[:1]:
    df_query = f"""
    WITH base_status AS (select
        station_id,
        hour,
        dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        status,
    """
    df_query += "".join([
        f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
        lead(num_bikes_available, {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) as remaining_bikes_available_{i},"""
     for i in list(range(1, 7)) + list(range(7, 18, 3))])
    df_query += f"""
    from
        status
    where
        year = 2022 and
        month = 10 and
        station_id = {station_id} and
        status = 'IN_SERVICE')"""
    df_query += " union ".join([
    f"""
    select
        station_id,
        hour,
        dow,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        minutes_bt_check_{i} as minutes_bt_check,
        remaining_bikes_available_{i} as remaining_bikes_available,
    from
        base_status
    """ for i in list(range(1, 7)) + list(range(7, 18, 3))])
    df2 = con.execute(df_query).df().dropna()
    
print(f"dataset size {len(df1)}")

dataset size 36662
CPU times: user 20.1 s, sys: 751 ms, total: 20.8 s
Wall time: 3.66 s


### Base select with leads + pandas dataframe rearrenge per lead

In [9]:
%%time 
for station_id in station_ids[:1]:
    df_query = f"""
    select
        station_id,
        hour,
        dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        status,
    """
    df_query += "".join([
        f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
        lead(num_bikes_available, {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) as remaining_bikes_available_{i},"""
     for i in list(range(1, 7)) + list(range(7, 18, 3))])
    df_query += f"""
    from
        status
    where
        year = 2022 and
        month = 10 and
        station_id = {station_id} and
        status = 'IN_SERVICE'"""
    df3 = con.execute(df_query).df()
dfs_to_concat = []
for i in list(range(1, 7)) + list(range(7, 18, 3)):
    dfs_to_concat.append(df3[["station_id", "hour", "dow", "num_bikes_available", "num_bikes_disabled", "num_docks_available",
                             "num_docks_disabled", "status",f"minutes_bt_check_{i}",
                             f"remaining_bikes_available_{i}"]].rename(columns={f"minutes_bt_check_{i}": "minutes_bt_check",
                                                                             f"remaining_bikes_available_{i}": "remaining_bikes_available"}))

pd.concat(dfs_to_concat).dropna().drop_duplicates()

print(f"dataset size {len(df1)}")

dataset size 36662
CPU times: user 1.94 s, sys: 95.3 ms, total: 2.04 s
Wall time: 591 ms


## Multiple stations

### Get all stations in one query

Causes kernel crash because runs out of ram.

In [6]:
%%time
df_query = f"""
select
    station_id,
    hour,
    dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
    num_bikes_available,
    num_bikes_disabled,
    num_docks_available,
    num_docks_disabled,
    status,
"""
df_query += "".join([
    f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
        partition by station_id
        order by make_timestamp(year, month, day, hour, minute, 0.0) asc
    ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
    lead(num_bikes_available, {i}) over (
        partition by station_id
        order by make_timestamp(year, month, day, hour, minute, 0.0) asc
    ) as remaining_bikes_available_{i},"""
    for i in list(range(1, 7)) + list(range(7, 18, 3))])
df_query += f"""
from
    status
where
    year = 2022 and
    month = 10 and
    status = 'IN_SERVICE'"""
df_complete = con.execute(df_query).df()

dfs_to_concat = []
for i in list(range(1, 7)) + list(range(7, 18, 3)):
    dfs_to_concat.append(df_complete[["station_id", "hour", "dow", "num_bikes_available", "num_bikes_disabled", "num_docks_available",
                             "num_docks_disabled", "status",f"minutes_bt_check_{i}",
                             f"remaining_bikes_available_{i}"]].rename(columns={f"minutes_bt_check_{i}": "minutes_bt_check",
                                                                             f"remaining_bikes_available_{i}": "remaining_bikes_available"}))

df_complete = pd.concat(dfs_to_concat).dropna().drop_duplicates()
print(f"dataset size {len(df_complete)}")

: 

: 

### Iterate per station and concat dataframes

In [13]:
%%time
df_complete = []
for station_id in station_ids[:10]:
    df_query = f"""
    select
        station_id,
        hour,
        dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
        num_bikes_available,
        num_bikes_disabled,
        num_docks_available,
        num_docks_disabled,
        status,
    """
    df_query += "".join([
        f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
        lead(num_bikes_available, {i}) over (
            order by make_timestamp(year, month, day, hour, minute, 0.0) asc
        ) as remaining_bikes_available_{i},"""
     for i in list(range(1, 7)) + list(range(7, 18, 3))])
    df_query += f"""
    from
        status
    where
        year = 2022 and
        month = 10 and
        station_id = {station_id} and
        status = 'IN_SERVICE'"""
    df_complete.append(con.execute(df_query).df())

df_complete = pd.concat(df_complete)
dfs_to_concat = []
for i in list(range(1, 7)) + list(range(7, 18, 3)):
    dfs_to_concat.append(df_complete[["station_id", "hour", "dow", "num_bikes_available", "num_bikes_disabled", "num_docks_available",
                             "num_docks_disabled", "status",f"minutes_bt_check_{i}",
                             f"remaining_bikes_available_{i}"]].rename(columns={f"minutes_bt_check_{i}": "minutes_bt_check",
                                                                             f"remaining_bikes_available_{i}": "remaining_bikes_available"}))

df_complete=pd.concat(dfs_to_concat).dropna().drop_duplicates()
print(f"dataset size {len(df_complete)}")

dataset size 408087
CPU times: user 18.3 s, sys: 1.4 s, total: 19.7 s
Wall time: 5.27 s


### Get multiple stations with IN

In [14]:
%%time
df_query = f"""
select
    station_id,
    hour,
    dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
    num_bikes_available,
    num_bikes_disabled,
    num_docks_available,
    num_docks_disabled,
    status,
"""
df_query += "".join([
    f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
        partition by station_id
        order by make_timestamp(year, month, day, hour, minute, 0.0) asc
    ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
    lead(num_bikes_available, {i}) over (
        partition by station_id
        order by make_timestamp(year, month, day, hour, minute, 0.0) asc
    ) as remaining_bikes_available_{i},"""
    for i in list(range(1, 7)) + list(range(7, 18, 3))])
df_query += f"""
from
    status
where
    year = 2022 and
    month = 10 and
    station_id IN (
"""
df_query += " , ".join([str(s) for s in station_ids[:10]])
df_query += """) and
    status = 'IN_SERVICE'"""

df_complete = con.execute(df_query).df()
dfs_to_concat = []
for i in list(range(1, 7)) + list(range(7, 18, 3)):
    dfs_to_concat.append(df_complete[["station_id", "hour", "dow", "num_bikes_available", "num_bikes_disabled", "num_docks_available",
                             "num_docks_disabled", "status",f"minutes_bt_check_{i}",
                             f"remaining_bikes_available_{i}"]].rename(columns={f"minutes_bt_check_{i}": "minutes_bt_check",
                                                                             f"remaining_bikes_available_{i}": "remaining_bikes_available"}))

df_complete=pd.concat(dfs_to_concat).dropna().drop_duplicates()
print(f"dataset size {len(df_complete)}")

dataset size 408087
CPU times: user 5.03 s, sys: 576 ms, total: 5.61 s
Wall time: 2.74 s


In [8]:
%%time
batch_size = 100

with tempfile.TemporaryDirectory() as tmpdirname:
    print(tmpdirname)
    for batch_start in range(0, len(station_ids), batch_size):
        df_query = f"""
        select
            station_id,
            hour,
            dayofweek(make_timestamp(year, month, day, hour, minute, 0.0)) as dow,
            num_bikes_available,
            num_bikes_disabled,
            num_docks_available,
            num_docks_disabled,
            status,
        """
        df_query += "".join([
            f"""minute(lead(make_timestamp(year, month, day, hour, minute, 0.0), {i}) over (
                partition by station_id
                order by make_timestamp(year, month, day, hour, minute, 0.0) asc
            ) - make_timestamp(year, month, day, hour, minute, 0.0)) as minutes_bt_check_{i},
            lead(num_bikes_available, {i}) over (
                partition by station_id
                order by make_timestamp(year, month, day, hour, minute, 0.0) asc
            ) as remaining_bikes_available_{i},"""
            for i in list(range(1, 7)) + list(range(7, 18, 3))])
        df_query += f"""
        from
            status
        where
            year = 2022 and
            month = 10 and
            station_id IN (
        """
        df_query += " , ".join([str(s) for s in station_ids[batch_start:batch_start+batch_size]])
        df_query += """) and
            status = 'IN_SERVICE'"""

        df_complete=con.execute(df_query).df()

        dfs_to_concat = []
        for i in list(range(1, 7)) + list(range(7, 18, 3)):
            dfs_to_concat.append(df_complete[["station_id", "hour", "dow", "num_bikes_available", "num_bikes_disabled", "num_docks_available",
                                    "num_docks_disabled", "status",f"minutes_bt_check_{i}",
                                    f"remaining_bikes_available_{i}"]].rename(columns={f"minutes_bt_check_{i}": "minutes_bt_check",
                                                                                    f"remaining_bikes_available_{i}": "remaining_bikes_available"}))

        del df_complete
        pd.concat(dfs_to_concat).dropna().drop_duplicates().to_csv(tmpdirname+f"/{batch_start}.csv")

    dfs_to_concat = []
    for batch_start in range(0, len(station_ids), batch_size):
        dfs_to_concat.append(pd.read_csv(tmpdirname+f"/{batch_start}.csv"))

    df_complete = pd.concat(dfs_to_concat)

print(f"dataset size {len(df_complete)}")

/tmp/tmp5k0yln6l
dataset size 9301095
CPU times: user 1min 53s, sys: 13.4 s, total: 2min 7s
Wall time: 1min 40s


In [10]:
df_complete.to_csv("avail_complete.csv")